In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = "/Apple-Invariant-feature"
os.chdir(path)
os.getcwd()

In [ ]:
!pip install pyradiomics

In [ ]:
import pathlib
import imageio.v2 as imageio
import SimpleITK as sitk
import matplotlib.pyplot as plt
from radiomics import featureextractor
import pandas as pd
import re
import numpy as np
from radiomics import glszm

In [ ]:
directory = pathlib.Path("/Apple-Invariant-feature/ApplePost")

all_images = []

for file in directory.iterdir():
    if file.is_file():
        file_name = file.name
        if file_name.lower().endswith(".png"):
          if not file_name.startswith("mask"):
            all_images.append(file_name)

print(len(all_images))

In [ ]:
#extract all original texture features from pyradiomics
dir = "/Apple-Invariant-feature/ApplePost"
extractor = featureextractor.RadiomicsFeatureExtractor()
radiomic_features = {}

for folder_name in all_images:
  image_path = dir + "/" + folder_name
  mask_path = dir + "/mask_" + folder_name
  image = imageio.imread(image_path)
  mask = imageio.imread(mask_path)
  image = sitk.GetImageFromArray(image[:, :, 2])
  mask = sitk.GetImageFromArray((mask[:, :, 1] > 240).astype(np.uint8))
  result = extractor.execute(image, mask)
  radiomic_features[str(folder_name)] = result

radiomic_features = pd.DataFrame.from_dict(radiomic_features, orient='index')
radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='diagnostics')))]
radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='firstorder')))]

# Sort the DataFrame based on the index
radiomic_features = radiomic_features.sort_index()

# Save the sorted DataFrame to Excel
radiomic_features.to_excel('original_radiomic_features_Apple.xlsx')